In [ ]:
import os
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio import warp
from netCDF4 import Dataset
from osgeo import gdal
from datetime import datetime, timedelta
import pandas as pd
import glob
import calendar
import xcdat as xc
import xarray as xa


event = "event5"
event_dir = f"/p/vast1/lazin1/UNet_inputs/Geotiff_var/WM/Tile_Sonoma_hourly/{event}/"


files = sorted(glob.glob(f"{event_dir}/*.tif")) #'/p/lustre1/lazin1/RAPID_Archive_Flood_Maps/Tile_No_Threshold_Mississippi_20190617_5E5F_non_flood'
# reference_raster_files= reference_raster_files.to_numpy()


output_geotiff_dir = f"/p/vast1/lazin1/UNet_inputs/Geotiff_var/1H_prec/Sonoma/{event}/" #Harvey_D374'  #5E5F
os.makedirs(output_geotiff_dir, exist_ok=True)
path = '/p/lustre2/lazin1/AORC_APCP_surface/'


start_time = datetime.strptime("2006041200", "%Y%m%d%H")
end_time = datetime.strptime("2006041423", "%Y%m%d%H")
all_hours = pd.date_range(start=start_time, end=end_time, freq='h')

tile_csv = f"/usr/workspace/lazin1/anaconda_dane/envs/RAPID/Codes/Sonoma_shapefile/Sonoma_DEM_tiles.csv" #f"/usr/workspace/lazin1/anaconda_dane/envs/RAPID/Codes/Sonoma_shapefile/Tile_Sonoma_hourly_event2.csv"#f"/usr/workspace/lazin1/anaconda_dane/envs/RAPID/Codes/Sonoma_shapefile/Tile_Sonoma_hourly_event2_test.csv"
tiles = pd.read_csv(tile_csv)['ras_name'].tolist()

                
start_ym = start_time.strftime("%Y") + "_" + start_time.strftime("%m")
end_ym = end_time.strftime("%Y") + "_" + end_time.strftime("%m")



print(start_ym, end_ym)
file_pattern = "APCP_surface_*.nc"  # Modify if files are in a different directory
all_files = sorted(glob.glob(path+file_pattern))  # Get all matching files
print(len(all_files))
# Filter files based on year-month
filtered_files = []
for file in all_files:
    year = file.split("_")[-2]  
    month = file.split("_")[-1].replace(".nc", "")  # Extract YYYY_MM
    year_month = year + "_" + month

    if start_ym <= year_month <= end_ym:
        filtered_files.append(file)
print(filtered_files)

# Open and concatenate all selected files
precip = []

for f in filtered_files:
    print(f)
    precip_slice = xc.open_dataset(f)
    precip_slice = precip_slice.sel(latitude=slice(38.26, 39.5), longitude=slice(-123.5, -122.4))
    precip.append(precip_slice)
    # print(precip_slice)
    # print(precip_slice["time"])
precip = xa.concat(precip, dim='time')
_, index = np.unique(precip["time"], return_index=True)
precip = precip.isel(time=index)
precip = precip.sel(time=slice(start_time, end_time))
# print(precip)

# save to tiff:
data_var = precip['APCP_surface'].fillna(0)

data_var_flip = data_var.isel(latitude=slice(None, None, -1))  # Flip latitude
# print('flip',data_var_flip[:1,:5,:1])
# print('data_var',data_var[:1,:5,:1])

precip_1H= data_var.sum(dim="time")
# print(data_var_flip.shape, data_var.shape, precip_1day.shape)

transform = from_origin(
west=precip.longitude.min().values, 
north=precip.latitude.max().values, 
xsize=(precip.longitude[1] - precip.longitude[0]).values, 
ysize=(precip.latitude[1] - precip.latitude[0]).values
)
if len(all_hours) == data_var_flip.shape[0]:
    
    for i in range(data_var_flip.shape[0]):
        
        with rasterio.open(
            os.path.join(output_geotiff_dir, f"{event}_{all_hours[i].strftime('%Y%m%d%H')}_Sonoma.tif"), 
            "w",
            driver="GTiff",
            height=precip_1H.shape[0],
            width=precip_1H.shape[1],
            count=1,
            dtype="float32",
            crs="EPSG:4326",
            transform=transform
        ) as temp_dst:
            temp_dst.write((np.flip(precip_1H, axis=0)), 1)

2014_12 2014_12
244
['/p/lustre2/lazin1/AORC_APCP_surface/APCP_surface_2014_12.nc']
/p/lustre2/lazin1/AORC_APCP_surface/APCP_surface_2014_12.nc


/usr/workspace/lazin1/anaconda_dane/envs/RAPID/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


: 

In [1]:
import os
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio import warp
from netCDF4 import Dataset
from osgeo import gdal
from datetime import datetime, timedelta
import pandas as pd
import glob
import calendar
import xcdat as xc
import xarray as xa


event = "event4"
event_dir = f"/p/vast1/lazin1/UNet_inputs/Geotiff_var/WM/Tile_Sonoma_hourly/{event}/"


files = sorted(glob.glob(f"{event_dir}/*.tif")) #'/p/lustre1/lazin1/RAPID_Archive_Flood_Maps/Tile_No_Threshold_Mississippi_20190617_5E5F_non_flood'
# reference_raster_files= reference_raster_files.to_numpy()


output_geotiff_dir = f"/p/vast1/lazin1/UNet_inputs/Geotiff_var/1H_prec/Sonoma/{event}/" #Harvey_D374'  #5E5F
os.makedirs(output_geotiff_dir, exist_ok=True)
path = '/p/lustre2/lazin1/AORC_APCP_surface/'


start_time = datetime.strptime("2014121100", "%Y%m%d%H")
end_time = datetime.strptime("2014121223", "%Y%m%d%H")
all_hours = pd.date_range(start=start_time, end=end_time, freq='h')

# tile_csv = f"/usr/workspace/lazin1/anaconda_dane/envs/RAPID/Codes/Sonoma_shapefile/Sonoma_DEM_tiles.csv" #f"/usr/workspace/lazin1/anaconda_dane/envs/RAPID/Codes/Sonoma_shapefile/Tile_Sonoma_hourly_event2.csv"#f"/usr/workspace/lazin1/anaconda_dane/envs/RAPID/Codes/Sonoma_shapefile/Tile_Sonoma_hourly_event2_test.csv"
DEM_dir = f"/p/vast1/lazin1/UNet_inputs/Geotiff_var/cropped_DEM/Sonoma"
tiles = sorted(glob.glob(f"{DEM_dir}/*.tif"))


for h in all_hours:
    temp_raster_path = os.path.join(output_geotiff_dir, f"{event}_{h.strftime('%Y%m%d%H')}_Sonoma.tif")
    # temp_raster_path = os.path.join(output_geotiff_dir, f.split("/")[-1].split("crop")[0][:-1] + '.tif')
    print(temp_raster_path)
    for tile in tiles:
        output_geotiff = os.path.join(output_geotiff_dir, f"1H_prec_{event}_{h.strftime('%Y%m%d%H')}_{tile.split("/")[-1]}")
        # output_geotiff = f"{output_geotiff_dir}/1H_prec_"+ f.split("/")[-1]
        # print(output_geotiff)
        
        
        # temp_raster_path = os.path.join(output_geotiff_dir, reference_raster_file[1].split("crop")[0][:-1]+'.tif')
        # output_geotiff = os.path.join(output_geotiff_dir, '1day_prec_' + reference_raster_file[1])
        # reference_raster = f"{DEM_dir}/{tile}" #os.path.join(reference_raster_dir,reference_raster_file[1])
        reference_raster = tile
        with rasterio.open(reference_raster) as ref:
            ref_transform = ref.transform
            ref_crs = ref.crs
            ref_shape = (ref.height, ref.width)
            ref_bounds = ref.bounds
            
        with rasterio.open(temp_raster_path) as src:
            with rasterio.open(
                output_geotiff,
                "w",
                driver="GTiff",
                height=ref_shape[0],
                width=ref_shape[1],
                count=1,
                dtype="float32",
                crs=ref_crs,
                transform=ref_transform,
                nodata=-9999
            ) as dst:
                # Resample and reproject the data
                warp.reproject(
                    source=rasterio.band(src, 1),
                    destination=rasterio.band(dst, 1),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=ref_transform,
                    dst_crs=ref_crs,
                    resampling=warp.Resampling.nearest
                )

/p/vast1/lazin1/UNet_inputs/Geotiff_var/1H_prec/Sonoma/event4/event4_2014121100_Sonoma.tif
/p/vast1/lazin1/UNet_inputs/Geotiff_var/1H_prec/Sonoma/event4/event4_2014121101_Sonoma.tif
/p/vast1/lazin1/UNet_inputs/Geotiff_var/1H_prec/Sonoma/event4/event4_2014121102_Sonoma.tif
/p/vast1/lazin1/UNet_inputs/Geotiff_var/1H_prec/Sonoma/event4/event4_2014121103_Sonoma.tif
/p/vast1/lazin1/UNet_inputs/Geotiff_var/1H_prec/Sonoma/event4/event4_2014121104_Sonoma.tif
/p/vast1/lazin1/UNet_inputs/Geotiff_var/1H_prec/Sonoma/event4/event4_2014121105_Sonoma.tif
/p/vast1/lazin1/UNet_inputs/Geotiff_var/1H_prec/Sonoma/event4/event4_2014121106_Sonoma.tif
/p/vast1/lazin1/UNet_inputs/Geotiff_var/1H_prec/Sonoma/event4/event4_2014121107_Sonoma.tif
/p/vast1/lazin1/UNet_inputs/Geotiff_var/1H_prec/Sonoma/event4/event4_2014121108_Sonoma.tif
/p/vast1/lazin1/UNet_inputs/Geotiff_var/1H_prec/Sonoma/event4/event4_2014121109_Sonoma.tif
/p/vast1/lazin1/UNet_inputs/Geotiff_var/1H_prec/Sonoma/event4/event4_2014121110_Sonoma.tif